# INST326 Week 10 Lecture
## Composition vs Inheritance: Architectural Decision-Making

**Duration:** 75 minutes (1:15)  
**Date:** November 17, 2025

---

## Learning Objectives
By the end of this lecture, you will be able to:
- Distinguish between "is-a" (inheritance) and "has-a" (composition) relationships
- Apply design principles to choose between inheritance and composition
- Implement composition patterns using the Cell class in the garden system
- Evaluate the trade-offs between different architectural approaches
- Recognize when to favor composition over inheritance in Information Science applications
- Connect architectural decisions to the broader pattern thinking that defines professional development

---

## The Big Picture: Why This Week Matters

**You've learned powerful tools:**
- Week 8: Inheritance lets you create specialized classes
- Week 9: Polymorphism lets you treat different classes uniformly

**But with great power comes important decisions:**
- When should you use inheritance?
- When should you use composition instead?
- How do these decisions affect your system's flexibility and maintainability?

**This is pattern thinking in action** - the human skill that distinguishes thoughtful architecture from code generation.

---
# Part 1: Composition vs Inheritance Fundamentals
## (45 minutes)

### The Two Fundamental Relationships (10 minutes)

**Every object relationship is either:**
1. **IS-A** (inheritance): A RaisedBed **is a** PlantingContainer
2. **HAS-A** (composition): A Cell **has a** PlantingContainer

**The question:** Which relationship better represents your domain?

In [1]:
# Review: Inheritance (IS-A relationship)
# We've already seen this pattern

class PlantingContainer:
    """Base class for all planting containers."""
    def __init__(self, length, width, depth):
        self.length = length
        self.width = width
        self.depth = depth

    def calculate_area(self):
        """Calculate planting surface area."""
        return self.length * self.width

    def calculate_volume(self):
        """Calculate soil volume."""
        return self.length * self.width * self.depth

class RaisedBed(PlantingContainer):
    """A RaisedBed IS-A PlantingContainer."""
    def __init__(self, length, width, depth, material):
        super().__init__(length, width, depth)
        self.material = material

    def calculate_lumber_needed(self):
        """RaisedBed-specific method."""
        perimeter = 2 * (self.length + self.width)
        return perimeter * self.depth

# Test it
bed = RaisedBed(48, 24, 12, "cedar")
print(f"Bed area: {bed.calculate_area()} sq in")
print(f"Lumber needed: {bed.calculate_lumber_needed()} board inches")
print(f"A RaisedBed is a PlantingContainer: {isinstance(bed, PlantingContainer)}")

Bed area: 1152 sq in
Lumber needed: 1728 board inches
A RaisedBed is a PlantingContainer: True


### Introducing Composition (HAS-A relationship) (15 minutes)

**Now let's look at a different kind of relationship:**

A **Cell** is a plantable section within a container. It doesn't "specialize" a container - it **uses** one.

**Key insight:**
- A Cell is NOT a type of container
- A Cell CONTAINS a reference to a container
- A Cell also HAS a Plant growing in it
- A Cell HAS a location within the container

This is **composition** - building complex objects by combining simpler ones.

In [3]:
# First, let's define a simple Plant class for our example
class Plant:
    """Represents a plant in the garden."""
    def __init__(self, name, spacing_required, days_to_maturity):
        self.name = name
        self.spacing_required = spacing_required  # inches
        self.days_to_maturity = days_to_maturity

    def __repr__(self):
        return f"Plant('{self.name}')"

# Now: Cell with Composition (HAS-A relationships)
class Cell:
    """A Cell HAS-A PlantingContainer and HAS-A Plant.

    Notice: Cell does NOT inherit from PlantingContainer.
    Instead, it holds a REFERENCE to one.
    """
    def __init__(self, container, column, row, cell_size=12):
        # Composition: Cell HAS-A container
        self.container = container
        self.column = column  # Grid position (A, B, C, etc.)
        self.row = row        # Grid position (1, 2, 3, etc.)
        self.cell_size = cell_size  # Square foot gardening: 12x12 inches

        # Composition: Cell HAS-A plant (or None if empty)
        self.plant = None
        self.planted_date = None

    def get_location_id(self):
        """Return cell location like 'A1', 'B3', etc."""
        return f"{self.column}{self.row}"

    def plant_in_cell(self, plant, date):
        """Plant something in this cell."""
        if self.plant is not None:
            raise ValueError(f"Cell {self.get_location_id()} is already occupied by {self.plant.name}")

        # Check if plant fits
        if plant.spacing_required > self.cell_size:
            raise ValueError(f'{plant.name} requires {plant.spacing_required}" spacing')

        self.plant = plant
        self.planted_date = date

    def harvest(self):
        """Harvest and clear the cell."""
        if self.plant is None:
            raise ValueError(f"Cell {self.get_location_id()} is empty")

        harvested = self.plant
        self.plant = None
        self.planted_date = None
        return harvested

    def is_occupied(self):
        """Check if cell has a plant."""
        return self.plant is not None

    def get_container_info(self):
        """Access container information through composition."""
        return {
            'container_area': self.container.calculate_area(),
            'container_volume': self.container.calculate_volume(),
            'cell_location': self.get_location_id()
        }

    def __repr__(self):
        status = f"planted with {self.plant.name}" if self.plant else "empty"
        return f"Cell {self.get_location_id()} ({status})"

# Demonstrate composition
print("=== Creating Garden with Composition ===")
bed = RaisedBed(48, 24, 12, "cedar")
cell_a1 = Cell(bed, 'A', 1)
cell_b2 = Cell(bed, 'B', 2)

print(f"\nCreated cells in the same raised bed:")
print(cell_a1)
print(cell_b2)

# Plant something
tomato = Plant("Cherry Tomato", 12, 65)
lettuce = Plant("Buttercrunch Lettuce", 6, 55)

cell_a1.plant_in_cell(tomato, "2025-05-15")
cell_b2.plant_in_cell(lettuce, "2025-05-15")

print(f"\nAfter planting:")
print(cell_a1)
print(cell_b2)

print(f"\nCell A1 container info: {cell_a1.get_container_info()}")

=== Creating Garden with Composition ===

Created cells in the same raised bed:
Cell A1 (empty)
Cell B2 (empty)

After planting:
Cell A1 (planted with Cherry Tomato)
Cell B2 (planted with Buttercrunch Lettuce)

Cell A1 container info: {'container_area': 1152, 'container_volume': 13824, 'cell_location': 'A1'}


### Why Use Composition Here? (10 minutes)

**Question: Why didn't we make Cell inherit from PlantingContainer?**

Let's think about what would happen if we tried:

```python
# BAD DESIGN: Cell inheriting from PlantingContainer
class Cell(PlantingContainer):
    # Problem 1: What dimensions do we pass to PlantingContainer?
    # A cell is 12x12, but it exists WITHIN a larger container
    
    # Problem 2: A Cell IS-NOT-A PlantingContainer
    # It's a section WITHIN a container
    
    # Problem 3: Multiple cells share the same physical container
    # How do we represent that with inheritance?
```

**The Liskov Substitution Principle says:**
If S is a subtype of T, then objects of type T should be replaceable with objects of type S. (See SOLID Principles Summary https://claude.ai/public/artifacts/257c8750-5aee-404d-aa3a-5978ccc046e8)

**Applied here:**
- Can we use a Cell anywhere we use a PlantingContainer? NO!
- A Cell is not an independent container - it's part of one
- Therefore: Cell should NOT inherit from PlantingContainer

In [5]:
# Benefits of Composition: Flexibility

class GardenManager:
    """Manages multiple containers and their cells."""
    def __init__(self):
        self.containers = []  # List of PlantingContainer objects
        self.cells = []       # List of Cell objects

    def add_container(self, container):
        """Add a container to the garden."""
        self.containers.append(container)

    def create_cells_for_container(self, container, rows, columns):
        """Create a grid of cells for a container.

        This shows composition's flexibility: we can create
        any number of cells for any container dynamically.
        """
        column_letters = 'ABCDEFGHIJKLMNOPQRSTUVWXYZ'

        for row in range(1, rows + 1):
            for col_idx in range(columns):
                col_letter = column_letters[col_idx]
                cell = Cell(container, col_letter, row)
                self.cells.append(cell)

    def get_cell(self, location_id):
        """Find a cell by location ID like 'A1'."""
        for cell in self.cells:
            if cell.get_location_id() == location_id:
                return cell
        return None

    def get_occupied_cells(self):
        """Get all cells that have plants."""
        return [cell for cell in self.cells if cell.is_occupied()]

    def get_available_cells(self):
        """Get all empty cells."""
        return [cell for cell in self.cells if not cell.is_occupied()]

# Demonstrate flexibility
garden = GardenManager()

# Add different types of containers
big_bed = RaisedBed(48, 48, 12, "cedar")
small_bed = RaisedBed(24, 24, 10, "pine")

garden.add_container(big_bed)
garden.add_container(small_bed)

# Create different cell layouts for each
garden.create_cells_for_container(big_bed, rows=4, columns=4)  # 4x4 grid
garden.create_cells_for_container(small_bed, rows=2, columns=2)  # 2x2 grid

print(f"\nTotal cells created: {len(garden.cells)}")
print(f"Available for planting: {len(garden.get_available_cells())}")

# Plant in a few cells
basil = Plant("Sweet Basil", 6, 60)
pepper = Plant("Bell Pepper", 12, 75)

garden.get_cell('A1').plant_in_cell(basil, "2025-05-20")
garden.get_cell('D4').plant_in_cell(pepper, "2025-05-20")

print(f"\nAfter planting:")
print(f"Occupied cells: {len(garden.get_occupied_cells())}")
for cell in garden.get_occupied_cells():
    print(f"  {cell}")
print(f"Available for planting: {len(garden.get_available_cells())}")


Total cells created: 20
Available for planting: 20

After planting:
Occupied cells: 2
  Cell A1 (planted with Sweet Basil)
  Cell D4 (planted with Bell Pepper)
Available for planting: 18


### Decision Framework: Inheritance vs Composition (10 minutes)

**Use INHERITANCE (IS-A) when:**
1. There's a clear specialization relationship
   - A RaisedBed IS-A PlantingContainer (just a specific type)
2. The subclass truly can substitute for the parent
   - Anywhere you use a PlantingContainer, a RaisedBed works
3. You want to share implementation (DRY principle https://claude.ai/public/artifacts/757dda56-ee6c-47fb-a6da-458fca2b281f)
   - calculate_area() works for all container types
4. The relationship is stable and unlikely to change
   - A raised bed will always be a type of container

**Use COMPOSITION (HAS-A) when:**
1. Objects have a "part of" or "uses" relationship
   - A Cell HAS-A container (exists within one)
2. You need flexibility to change components
   - Could swap which container a cell refers to
3. Multiple objects might share the same component
   - Many cells reference the same container
4. The relationship is dynamic or optional
   - A cell might or might not have a plant
5. You want loose coupling between objects
   - Cell doesn't need to know container implementation details

**The "Favor Composition Over Inheritance" Principle:**
- When in doubt, use composition
- Composition is more flexible
- Inheritance creates tight coupling
- You can always add inheritance later if needed
- But removing inheritance is harder

---
# Part 2: Architectural Pattern Thinking
## (25 minutes)

### From Code to Patterns (12 minutes)

**This week reveals something profound about programming:**

You're not just learning when to use inheritance vs composition.
You're learning how to **think architecturally** - to see patterns in problems and choose appropriate solutions.

**This connects to the "through-line" of the course:**
- Week 1-3: You learned programming syntax (variables, loops, functions)
- Week 4-7: You learned OOP mechanisms (classes, encapsulation, methods)
- Week 8-9: You learned inheritance relationships
- **Week 10: You're learning to CHOOSE between design approaches**

This choice - inheritance vs composition - is a **pattern**. And recognizing patterns is the human skill that matters.

### Information Science Applications (8 minutes)

**Let's see this pattern in your project domains:**

#### Library Management System
```python
# Inheritance: Different types of library items
class LibraryItem:  # Parent
    pass

class Book(LibraryItem):     # IS-A LibraryItem
    pass

class Journal(LibraryItem):  # IS-A LibraryItem
    pass

# Composition: A loan HAS items and borrowers
class Loan:
    def __init__(self, item, borrower, due_date):
        self.item = item         # HAS-A LibraryItem
        self.borrower = borrower # HAS-A Member
        self.due_date = due_date
```

#### Digital Archive
```python
# Inheritance: Different document types
class ArchiveDocument:  # Parent
    pass

class Photograph(ArchiveDocument):  # IS-A Document
    pass

class Manuscript(ArchiveDocument):  # IS-A Document
    pass

# Composition: A collection HAS documents
class Collection:
    def __init__(self, name):
        self.name = name
        self.documents = []  # HAS-MANY Documents
```

#### Research Data Management
```python
# Inheritance: Different analysis types
class Analysis:  # Parent
    pass

class StatisticalAnalysis(Analysis):  # IS-A Analysis
    pass

class QualitativeAnalysis(Analysis):  # IS-A Analysis
    pass

# Composition: An experiment HAS datasets and analyses
class Experiment:
    def __init__(self, title):
        self.title = title
        self.datasets = []   # HAS-MANY DataSets
        self.analyses = []   # HAS-MANY Analyses
```

**The Pattern:**
- Use inheritance for type hierarchies (Book/Journal/DVD)
- Use composition for relationships (Loan contains Item and Member)
- Your system probably needs BOTH patterns working together

### Connection to Software Architecture (5 minutes)

**This isn't just about objects - it's about modular thinking:**

Recent research on "legible software" (Meng & Jackson, 2025) shows that:
- Good modularity means clear boundaries between components
- Composition creates looser coupling than inheritance
- Systems built with composition are easier for both humans AND AI to understand and extend

**Why this matters for your career:**
1. **LLMs can write code** - but they struggle with architectural decisions
2. **Pattern recognition** - knowing when to use which approach - is a human skill
3. **System thinking** - seeing how pieces fit together - is what makes you valuable

**The through-line revealed:**
- Programming isn't about syntax
- It's about recognizing patterns and making good architectural choices
- That's what this course has been teaching you all along

---
# Wrap-up & Preview
## (5 minutes)


### Next Week Preview:
- **Week 11 (Thanksgiving):** Design Patterns Overview
  - Factory, Strategy, Observer patterns
  - Lecture only - no lab or assignments
  - Time to catch up and reflect

### Looking Ahead to Project 4:
- **Week 12-13:** Capstone Integration & Testing
- You'll need both inheritance AND composition working together
- Professional testing and documentation
- Due 12/14

### Key Takeaway:
**Good architecture is about making thoughtful choices:**
- Not every relationship should be inheritance
- Not every relationship should be composition
- The skill is knowing which to use when
- This pattern thinking is what makes you a developer, not just a coder

---
# Instructor Notes

### Timing Flexibility:
- If composition concepts need more time, reduce IS examples section
- If students grasp composition quickly, expand on architectural patterns
- Save 5 minutes at end for Project 3 questions (it's due Sunday!)

### Interactive Elements:
- Poll: "When would YOU use inheritance vs composition in your project?"
- Live refactoring: Take an inheritance example and convert to composition
- Pair share: Students discuss their Project 3 design decisions

### ADHD-Friendly Structure:
- Clear sections with distinct examples
- Repeated pattern: Define → Demonstrate → Apply
- Visual contrast between inheritance tree and composition diagram
- Concrete before abstract (code examples before theory)

### Connection to Through-Line:
This is a pivotal lecture for revealing the course's deeper purpose:
- You're not teaching them to write code (AI can do that)
- You're teaching them to THINK about code architecturally
- Composition vs inheritance is a decision point that requires judgment
- This is pattern thinking - the human skill that matters

Consider previewing your Thanksgiving week lecture theme:
"Next week we'll see how the patterns you've learned - classes, inheritance, composition - combine into larger architectural patterns that professionals use across all domains."

### Assessment Connection:
- Exercise 10 directly practices composition with Cell class
- Project 3 (due Sunday!) requires both inheritance AND composition
- Discussion 10 asks students to reflect on their architectural choices
- This sets up Project 4 where they integrate everything

### Common Student Questions:
- "Can I use both in the same system?" → YES! (show Library example)
- "How do I know if I chose wrong?" → If it's hard to extend or modify
- "Does composition work with inheritance?" → Absolutely! (Cell + RaisedBed example)
- "What about Project 3?" → You probably need both patterns

### Reference Materials:
- Meng & Jackson paper on modularity (provided in documents)
- Bloch's "Favor composition over inheritance" (reading)
- Your own garden system as the primary teaching tool
- Student project examples from their chosen domains

### Project 3 Support:
Since Project 3 is due Sunday, students may have questions:
- Emphasize that most systems need BOTH patterns
- Inheritance for type hierarchies (Book/Journal, Vegetable/Herb)
- Composition for relationships (Collection contains Documents)
- Encourage them to sketch their object relationships
- Remind them of TA office hours this week

**This lecture bridges from OOP mechanics to OOP design thinking - a crucial transition for professional development.**